In [12]:
import pandas as pd
import numpy as np
from numpy.random import randint, shuffle
import random

def get_multiple_dfs(data_frame_rows):
    """
    Get a complete code str that creates a DF with random value
    """
    sizes_before_join = int(data_frame_rows * 1.1)
    start_with_offset = int(data_frame_rows * 0.1)
    end_with_offset = start_with_offset + sizes_before_join
    assert sizes_before_join - start_with_offset == data_frame_rows

    id_a = np.arange(sizes_before_join)
    shuffle(id_a)
    a = randint(0,100,size=(sizes_before_join))
    b = randint(0,100,size=(sizes_before_join))
    categories = ['cat_a', 'cat_b', 'cat_c']
    group_col_1 = pd.Series(random.choices(categories, k=sizes_before_join))
    group_col_2 = pd.Series(random.choices(categories, k=sizes_before_join))
    group_col_3 = pd.Series(random.choices(categories, k=sizes_before_join))

    id_b = np.arange(start_with_offset, end_with_offset)
    shuffle(id_b)
    c = randint(0,100,size=(sizes_before_join)) 
    d = randint(0,100,size=(sizes_before_join))

    df_a = pd.DataFrame(zip(id_a, a, b, group_col_1, group_col_2, group_col_3), columns=['id', 'A', 'B', 
        'group_col_1', 'group_col_2', 'group_col_3'])
    df_b = pd.DataFrame(zip(id_b, c, d), columns=['id', 'C', 'D'])
        
    return df_a, df_b

In [42]:
df_a, df_b = get_multiple_dfs(100000)

# Without instrumentation

In [72]:
%%time
test = df_a.merge(df_b, on='id')

CPU times: user 29.2 ms, sys: 9.25 ms, total: 38.4 ms
Wall time: 37.5 ms


In [52]:
from mlinspect.inspections._lineage import LineageId

lineage_id_list_a = [LineageId(0, row_id) for row_id in range(len(df_a))]
lineage_ids_a = pd.DataFrame({"RowLineage(10)": pd.Series(lineage_id_list_a, dtype="object")})
lineage_id_list_b = [LineageId(0, row_id) for row_id in range(len(df_b))]
lineage_ids_b = pd.DataFrame({"RowLineage(10)": pd.Series(lineage_id_list_b, dtype="object")})

In [59]:
from mlinspect.backends._pandas_backend import PandasBackend, execute_inspection_visits_join
from mlinspect.backends._pandas_backend_frame_wrapper import MlinspectDataFrame
from mlinspect.inspections import RowLineage
from mlinspect.inspections._inspection_input import OperatorContext
from mlinspect.instrumentation._dag_node import OperatorType, CodeReference

pandas_backend = PandasBackend()
pandas_backend.inspections = [RowLineage(10)]
function_info = ('pandas.core.frame', 'merge')
operator_context = OperatorContext(OperatorType.JOIN, function_info)
code_reference = CodeReference(0,0,0,10)

df_a = MlinspectDataFrame(df_a)
df_a.annotations = lineage_id_list_a
df_b = MlinspectDataFrame(df_b)
df_b.annotations = lineage_id_list_b

# With current instrumentation

In [75]:
%%time
df_a['mlinspect_index_x'] = range(0, len(df_a))
df_b['mlinspect_index_y'] = range(0, len(df_b))
original_return_value = df_a.merge(df_b, on='id')


return_value = execute_inspection_visits_join(pandas_backend, operator_context, code_reference,
                                              df_a,
                                              lineage_ids_a,
                                              df_b,
                                              lineage_ids_b,
                                              original_return_value)

# list(pandas_backend.dag_node_identifier_to_inspection_output.values())
# return_value

CPU times: user 1.07 s, sys: 56.1 ms, total: 1.12 s
Wall time: 1.12 s
